# Particionamento de tabelas  
  
\### Escopo  

<span style="font-size: 12pt; font-family: Arial, sans-serif; color: rgb(22, 22, 22); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Você precisa recomendar um método de particionamento para a tabela. A solução deve atender aos seguintes requisitos:</span>

- <span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Os dados devem ser mantidos por 2 anos.</span>
    
- <span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Todos os dados com mais de dois anos devem ser removidos uma vez por mês.</span>
    
- <span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">O desempenho das operações de remoção de dados deve ser maximizado.</span>
    

 <span style="color: var(--vscode-foreground);">O </span> **particionamento** <span style="color: var(--vscode-foreground);"> de tabelas é uma técnica usada para dividir grandes volumes de dados em partes menores e mais gerenciáveis, chamadas de </span> **partições**<span style="color: var(--vscode-foreground);">. No contexto do </span> **Azure SQL Database** <span style="color: var(--vscode-foreground);"> ou </span> **Azure SQL Managed Instance**<span style="color: var(--vscode-foreground);">, particionar tabelas pode melhorar a performance das consultas, facilitar a manutenção e otimizar o uso de recursos ao trabalhar com grandes conjuntos de dados.</span>

<span style="background-color: transparent; white-space-collapse: preserve; font-size: 14pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline;"><br></span>

### Scripts executados nessa configuração
Microsoft SQL Server 2022 (RTM-GDR) (KB5042211) - 16.0.1125.1 (X64)   Jul 31 2024 23:58:42   Copyright (C) 2022 Microsoft Corporation  Developer Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 22631: ) (Hypervisor) 


#### O que é o particionamento de tabelas?

# 

O particionamento envolve dividir uma tabela grande em segmentos menores (partições), baseando-se em uma coluna específica (como datas ou IDs). Cada partição pode ser armazenada de forma separada, mas elas ainda se comportam como uma única tabela do ponto de vista das consultas. Isso permite que o SQL Server processe as consultas de forma mais eficiente, acessando apenas as partições relevantes.

#### Vantagens do particionamento

# 

1. **Melhoria de desempenho**: Consultas que acessam uma pequena parte dos dados podem ser otimizadas, uma vez que o SQL Server só precisa ler as partições necessárias, reduzindo o I/O e o tempo de execução.
2. **Gerenciamento simplificado**: Tarefas de manutenção, como reconstrução de índices ou arquivamento de dados, podem ser realizadas em partições individuais, sem afetar a tabela inteira.
3. **Escalabilidade**: Ao particionar grandes volumes de dados, o armazenamento e a manutenção se tornam mais fáceis, permitindo que os bancos de dados cresçam de forma mais eficiente.
4. **Melhor uso de paralelismo**: O SQL Server pode distribuir operações entre diferentes partições e utilizar paralelismo de forma mais eficiente.

### Como funciona o particionamento no Azure SQL?

No Azure SQL, o particionamento segue a mesma abordagem do SQL Server local, usando **esquemas de partição** e **funções de partição**.

#### 1\. **Função de partição**

A função de partição define como os dados serão distribuídos entre as diferentes partições. Ela especifica os intervalos de valores que cada partição conterá.

Exemplo de criação de uma função de partição com base em uma coluna de data:  
  
CREATE PARTITION FUNCTION MyPartitionFunction (datetime) AS RANGE RIGHT FOR VALUES ('2023-01-01', '2023-06-01', '2023-12-01');

Neste exemplo, a função divide a tabela em quatro partições, baseando-se nos intervalos de datas.

#### 2\. **Esquema de partição**

O esquema de partição define em quais arquivos (ou grupos de arquivos) as partições serão armazenadas. No Azure SQL Database, o armazenamento é gerenciado automaticamente pelo serviço, então, normalmente, você atribui todas as partições ao mesmo grupo de arquivos.

Exemplo de criação de um esquema de partição:  
  
CREATE PARTITION SCHEME MyPartitionScheme AS PARTITION MyPartitionFunction ALL TO (\[PRIMARY\]);  
  
Este esquema associa todas as partições ao grupo de arquivos principal (`PRIMARY`).

#### 3\. **Criar a tabela particionada**

Ao criar uma tabela particionada, você utiliza a função e o esquema de partição para definir como os dados serão distribuídos nas partições.

Exemplo de criação de uma tabela particionada:

CREATE TABLE Sales
(
    SaleID INT PRIMARY KEY,
    SaleDate DATETIME,
    Amount DECIMAL(10, 2)
)
ON MyPartitionScheme(SaleDate);
  

#### Neste exemplo, a tabela `Sales` é particionada com base na coluna `SaleDate`, utilizando o esquema e a função de partição previamente definidos.

####

### Como o particionamento afeta consultas no Azure SQL?

Quando você consulta uma tabela particionada, o SQL Server pode otimizar o plano de execução, consultando apenas as partições relevantes, em vez de varrer toda a tabela. Isso é chamado de **eliminação de partições**.

Por exemplo, se sua tabela for particionada por data e você fizer uma consulta para buscar dados de vendas de junho de 2023:

<span style="color: #0000ff;">SELECT</span> <span style="color: #000000;">*</span> <span style="color: #0000ff;">FROM</span> Sales <span style="color: #0000ff;">WHERE</span> SaleDate <span style="color: #000000;">=</span> <span style="color: #a31515;">'2023-06-15'</span>;  
  
O SQL Server acessará apenas a partição correspondente a junho de 2023, ignorando todas as outras partições.

### Desafios e boas práticas

- **Chave de partição adequada**: Escolher uma boa coluna para ser a chave de partição é essencial. Colunas com alta cardinalidade (muitos valores distintos, como datas ou IDs sequenciais) são boas candidatas.
- **Reequilíbrio de partições**: Se os dados não forem distribuídos de forma equilibrada, uma partição pode acabar se tornando muito grande. Isso pode prejudicar o desempenho, então o particionamento deve ser bem planejado.
- **Manutenção**: Operações como reconstrução de índices podem ser aplicadas individualmente a partições, mas precisam ser planejadas para evitar sobrecarga em partições maiores.

### Considerações no Azure SQL Database

### 

1. **Azure SQL Database** tem suporte para particionamento, mas não para particionamento em múltiplos discos ou grupos de arquivos, já que o armazenamento é abstraído no serviço.
2. Em **Azure SQL Managed Instance**, você tem mais flexibilidade, como suporte a grupos de arquivos e opções mais avançadas de particionamento.

# Caso de uso 1  Tabela particionada por Ano

### 1\. Criar O banco de Dados

In [1]:
Create Database DemoTablesPartition

Commands completed successfully.

Total execution time: 00:00:00.186

In [2]:
use  DemoTablesPartition

Commands completed successfully.

Total execution time: 00:00:00

A função de partição define como os dados serão distribuídos entre as partições. Neste caso, vamos criar uma função de partição que divide os dados por ano

### 1\. Criar a Função de Partição

A função de partição define como os dados serão distribuídos entre as partições. Neste caso, vamos criar uma função de partição que divide os dados por ano.

In [13]:
CREATE PARTITION FUNCTION YearPartitionFunction (DATE)
AS RANGE RIGHT FOR VALUES 
('2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01');



Commands completed successfully.

Total execution time: 00:00:00.001

### 2\. Criar o Esquema de Partição

O esquema de partição define onde cada partição será armazenada. Para simplificar, vamos armazenar todas as partições no mesmo grupo de arquivos.

In [14]:
CREATE PARTITION SCHEME YearPartitionScheme
AS PARTITION YearPartitionFunction
ALL TO ([PRIMARY]);


Partition scheme 'YearPartitionScheme' has been created successfully. 'PRIMARY' is marked as the next used filegroup in partition scheme 'YearPartitionScheme'.

Total execution time: 00:00:00.001

### 3\. Criar a Tabela Particionada

Agora, você pode criar a tabela que usará o esquema de partição definido. Vamos supor que queremos uma tabela para armazenar vendas, onde cada venda é associada a um ano.

In [22]:
CREATE TABLE Pedidos
(
    ID_Pedido VARCHAR(20),
    Data_Pedido DATE,
    ID_Cliente VARCHAR(20),
    Segmento VARCHAR(50),
    Regiao VARCHAR(50),
    Pais VARCHAR(50),
    Product_ID VARCHAR(20),
    Categoria VARCHAR(50),
    SubCategoria VARCHAR(50),
    Total_Vendas DECIMAL(18, 2),
    Quantidade INT,
    Desconto DECIMAL(18, 2),
    Lucro DECIMAL(18, 2),
    Prioridade VARCHAR(20),
    PRIMARY KEY (ID_Pedido, Data_Pedido)  -- Incluindo Data_Pedido na chave primária
)
ON YearPartitionScheme(Data_Pedido);


: Msg 2714, Level 16, State 6, Line 1
There is already an object named 'Pedidos' in the database.

Total execution time: 00:00:00.002

# Inserir Dados

## Quando você inserir dados na tabela `Pedidos`, o SQL Server automaticamente colocará os registros na partição correta com base no ano extraído da coluna `DataPedido`.

In [28]:
BULK INSERT dbo.Pedidos
FROM 'D:\wesley\2.Myrepos\Scripts_SQL_Server\3.DP-300- Banco de Dados na Nuvem\2. Particionamento de tabelas\arquivo\dataset790MIL.csv'
WITH
(
    FIELDTERMINATOR = ';',  -- Delimitador de campos
    ROWTERMINATOR = '\n',   -- Delimitador de linhas
    FIRSTROW = 2            -- Ignorar cabeçalho (opcional)
);


: Msg 4864, Level 16, State 1, Line 1
Bulk load data conversion error (type mismatch or invalid character for the specified codepage) for row 2, column 2 (Data_Pedido).

: Msg 4864, Level 16, State 1, Line 1
Bulk load data conversion error (type mismatch or invalid character for the specified codepage) for row 3, column 10 (Total_Vendas).

: Msg 4864, Level 16, State 1, Line 1
Bulk load data conversion error (type mismatch or invalid character for the specified codepage) for row 4, column 2 (Data_Pedido).

: Msg 4864, Level 16, State 1, Line 1
Bulk load data conversion error (type mismatch or invalid character for the specified codepage) for row 5, column 2 (Data_Pedido).

: Msg 4864, Level 16, State 1, Line 1
Bulk load data conversion error (type mismatch or invalid character for the specified codepage) for row 6, column 10 (Total_Vendas).

...

: Msg 4864, Level 16, State 1, Line 1
Bulk load data conversion error (type mismatch or invalid character for the specified codepage) for row 12, column 10 (Total_Vendas).

: Msg 4865, Level 16, State 1, Line 1
Cannot bulk load because the maximum number of errors (10) was exceeded.

: Msg 7399, Level 16, State 1, Line 1
The OLE DB provider "BULK" for linked server "(null)" reported an error. The provider did not give any information about the error.

: Msg 7330, Level 16, State 2, Line 1
Cannot fetch a row from OLE DB provider "BULK" for linked server "(null)".

Total execution time: 00:00:00.006

# Explicação dos Parâmetros

- **FROM**: O caminho para o arquivo CSV que você deseja importar.
- **FIELDTERMINATOR**: O caractere que separa os campos no arquivo. No seu caso, parece ser `;`.
- **ROWTERMINATOR**: O caractere que indica o final de uma linha. `\n` é comum para arquivos de texto.
- **FIRSTROW**: Este parâmetro é opcional. Se o seu arquivo CSV contém um cabeçalho (nomes das colunas), você pode definir `FIRSTROW = 2` para ignorar a primeira linha.

# 5. Consultar Dados

Você pode consultar a tabela normalmente, e o SQL Server cuidará da eliminação de partições, otimizando o acesso aos dados.

In [ ]:
SELECT * FROM Sales WHERE SaleDate >= '2021-01-01' AND SaleDate < '2022-01-01';


# Ajustar Partições

### Conforme você adiciona mais anos, pode ser necessário ajustar a função de partição e o esquema. Para adicionar um novo ano, você precisaria modificar a função de partição e, possivelmente, recriar o esquema ou utilizar comandos para adicionar partições

# <span style="font-size: 16pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(232, 232, 232); font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Caso de uso&nbsp;</span> 

  

### <span style="font-size: 13pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: rgb(232, 232, 232); font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Caso de Uso Prático: Sistema de Gestão de Documentos com FileTable</span>

<span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Imagine que sua empresa tem um </span> <span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">sistema de gestão de documentos</span> <span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;"> onde os funcionários precisam armazenar e acessar documentos, como contratos, relatórios, planilhas e outros arquivos de maneira centralizada. O objetivo é permitir que esses documentos sejam:</span>

1. <span style="font-size: 12pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Armazenados no banco de dados SQL Server</span> <span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;"> para controle centralizado e fácil backup.</span>
    
2. <span style="font-size: 12pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Acessados e manipulados diretamente via Windows Explorer</span> <span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;"> ou outra interface de sistema de arquivos, sem que o usuário precise interagir diretamente com o banco de dados.</span>
    
3. <span style="font-size: 12pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Consultados e gerenciados via T-SQL</span><span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">, possibilitando integração com outras funcionalidades e relatórios no sistema.</span>

## <span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">O </span> <span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">FileTable</span> <span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;"> é uma excelente solução para esse caso, já que permite armazenar esses documentos no SQL Server e mantê-los acessíveis pelo sistema de arquivos.</span>

<span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;"><br></span>

### <span style="font-size:13pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:700;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Passos para Implementar o FileTable:</span>

#### <span style="font-size:13pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:700;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">1. Pré-requisitos:</span>

### <span style="font-size:13pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:700;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;"><b style="font-weight:normal;" id="docs-internal-guid-ea4abfd6-7fff-c927-17c0-4207130e7c1f"><ul style="margin-top:0;margin-bottom:0;padding-inline-start:48px;"><li dir="ltr" style="list-style-type:disc;font-size:12pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;" aria-level="1"><p dir="ltr" style="line-height:1.2;margin-top:12pt;margin-bottom:0pt;" role="presentation"><span style="font-size:12pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">SQL Server 2012 ou posterior com suporte ao recurso </span><span style="font-size:12pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:700;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">FileTable</span><span style="font-size:12pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">.</span></p></li><li dir="ltr" style="list-style-type:disc;font-size:12pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;" aria-level="1"><p dir="ltr" style="line-height:1.2;margin-top:0pt;margin-bottom:12pt;" role="presentation"><span style="font-size:12pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Habilitar </span><span style="font-size:12pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:700;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">FILESTREAM</span><span style="font-size:12pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;"> no SQL Server, pois o FileTable depende dele.</span></p></li></ul></b></span>

#### <span style="font-size:13pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:700;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">2. Habilitar FILESTREAM no SQL Server:</span>

### <span style="font-size:13pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:700;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;"><b style="font-weight:normal;"><p dir="ltr" style="line-height:1.2;margin-top:12pt;margin-bottom:12pt;"><span style="font-size:12pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Você precisa habilitar o FILESTREAM tanto no nível de servidor quanto no nível de banco de dados.</span></p></b></span>

#### <span style="font-size:13.999999999999998pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:700;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">a) Habilitar FILESTREAM no nível de servidor:</span>

### <span style="font-size:13pt;font-family:'Times New Roman',serif;color:#000000;background-color:transparent;font-weight:700;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;"><span style="font-weight:normal;"><p dir="ltr" style="line-height:1.2;margin-top:12pt;margin-bottom:12pt;"><span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline;">Abra o SQL Server Configuration Manager e siga os passos:</span></p><ol style="margin-top:0;margin-bottom:0;padding-inline-start:48px;"><li dir="ltr" style="list-style-type: decimal; font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: nowrap;" aria-level="1"><p dir="ltr" style="line-height:1.2;margin-top:12pt;margin-bottom:0pt;" role="presentation"><span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">No </span><span style="font-size: 12pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">SQL Server Configuration Manager</span><span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">, selecione o seu servidor SQL e clique em "Propriedades".</span></p></li><li dir="ltr" style="list-style-type: decimal; font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: nowrap;" aria-level="1"><p dir="ltr" style="line-height:1.2;margin-top:0pt;margin-bottom:0pt;" role="presentation"><span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Na aba </span><span style="font-size: 12pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">FILESTREAM</span><span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">, marque as opções para habilitar o FILESTREAM:</span></p></li><ul style="margin-top:0;margin-bottom:0;padding-inline-start:48px;"><li dir="ltr" style="list-style-type: circle; font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: nowrap;" aria-level="2"><p dir="ltr" style="line-height:1.2;margin-top:0pt;margin-bottom:0pt;" role="presentation"><span style="font-size: 12pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Enable FILESTREAM for Transact-SQL access</span><span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">.</span></p></li><li dir="ltr" style="list-style-type: circle; font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: nowrap;" aria-level="2"><p dir="ltr" style="line-height:1.2;margin-top:0pt;margin-bottom:0pt;" role="presentation"><span style="font-size: 12pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Enable FILESTREAM for file I/O streaming access</span><span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">.</span></p></li><li dir="ltr" style="list-style-type: circle; font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: nowrap;" aria-level="2"><p dir="ltr" style="line-height:1.2;margin-top:0pt;margin-bottom:12pt;" role="presentation"><span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Marque também a opção para permitir acesso via </span><span style="font-size: 12pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Windows</span><span style="font-size: 12pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">.</span></p></li></ul></ol><br></span></span>

![Imagem de Exemplo](imagens/FileTable.png)


# Configurar FILESTREAM no banco de dados

## Agora que o FILESTREAM está habilitado no servidor, você deve configurá-lo no banco de dados. Execute os seguintes comandos SQL



In [ ]:
EXEC sp_configure filestream_access_level, 2;
RECONFIGURE;

# Criar um banco de dados com FILESTREAM e FileTable habilitados

In [5]:
CREATE DATABASE FileStreamDocumentManagement
ON PRIMARY 
(
    NAME = DocumentManagement_Data,
    FILENAME = 'D:\wesley\SQLDemo\FileStreamDocumentManagement.mdf'
),
FILEGROUP FileStreamGroup CONTAINS FILESTREAM 
(
    NAME =FileStreamDocumentManagement_FileStream,
    FILENAME = 'D:\wesley\SQLDemo\DocumentManagement_FS'
)
LOG ON 
(
    NAME = FileStreamDocumentManagement_Log,
    FILENAME = 'D:\wesley\SQLDemo\DocumentManagement_Log.ldf'
);



Query was canceled by user

Total execution time: 00:00:00.004

## Vamos conectar no banco de dados

In [6]:
USe FileStreamDocumentManagement

Query was canceled by user

Total execution time: 00:00:00

# Habilitar FileTable no banco de dados:
## Depois de criar o banco de dados, você precisa habilitar o suporte ao FileTable.

### SET FILESTREAM (NON\_TRANSACTED\_ACCESS = FULL) Aqui estamos configurando o comportamento do FILESTREAM para permitir acesso não transacional completo.

### NON_TRANSACTED_ACCESS
Esse parâmetro especifica se os dados FILESTREAM podem ser acessados fora do controle transacional do SQL Server, ou seja, através de chamadas ao sistema de arquivos diretamente, sem a necessidade de transações SQL.

In [7]:
ALTER DATABASE FileStreamDocumentManagement
SET FILESTREAM (NON_TRANSACTED_ACCESS = FULL);

ALTER DATABASE FileStreamDocumentManagement
SET ANSI_NULL_DEFAULT ON, ANSI_PADDING ON, ANSI_WARNINGS ON, ARITHABORT ON, CONCAT_NULL_YIELDS_NULL ON, QUOTED_IDENTIFIER ON, NUMERIC_ROUNDABORT OFF;


Query was canceled by user

Total execution time: 00:00:00

# Criar uma FileTable


##  Passo 1)

In [8]:
ALTER DATABASE FileStreamDocumentManagement
SET FILESTREAM (DIRECTORY_NAME = 'FileStreamDirectory');



Query was canceled by user

Total execution time: 00:00:00

## Passo 2)

In [13]:
CREATE TABLE EmployeeDocuments AS FileTable
WITH 
(
    FileTable_Directory = 'EmployeeFiles',  -- O nome da pasta no sistema de arquivos
    FileTable_Collate_Filename = DATABASE_DEFAULT
);



Query was canceled by user

Total execution time: 00:00:00

### Vamos analisar o comando em detalhes:

#### 1\. **CREATE TABLE EmployeeDocuments AS FileTable**

- **CREATE TABLE**: Inicia a criação de uma nova tabela no banco de dados.
- **EmployeeDocuments**: Este é o nome da tabela que está sendo criada. A tabela é chamada `EmployeeDocuments`.
- **AS FileTable**: Define que esta tabela será uma **FileTable**, que é uma tabela especial projetada para armazenar arquivos e documentos diretamente no sistema de arquivos, mantendo a capacidade de gerenciamento dentro do SQL Server.

#### 2\. **WITH (FileTable\_Directory = 'EmployeeFiles')**

- A cláusula **WITH** especifica opções adicionais para a FileTable.
- **FileTable\_Directory = 'EmployeeFiles'**: Este parâmetro define o nome da pasta no sistema de arquivos que será associada a essa FileTable. Neste caso, a pasta será chamada `EmployeeFiles`. Quando você adiciona arquivos à FileTable, eles serão fisicamente armazenados no diretório do sistema de arquivos correspondente, e a pasta será criada automaticamente.
    - Essa pasta está localizada dentro do diretório raiz do FILESTREAM que foi configurado para a instância do SQL Server.
    - Os arquivos adicionados à FileTable podem ser acessados diretamente como se estivessem em um sistema de arquivos normal, mas são gerenciados dentro do banco de dados.

#### 3\. **FileTable\_Collate\_Filename = DATABASE\_DEFAULT**

- **FileTable\_Collate\_Filename** define a regra de ordenação (collation) usada para os nomes dos arquivos na FileTable.
- **DATABASE\_DEFAULT**: Especifica que o padrão de collation para os nomes dos arquivos deve ser o mesmo utilizado no banco de dados. Isso garante que os nomes de arquivos armazenados na FileTable seguirão as regras de ordenação e sensibilidade a maiúsculas e minúsculas do banco de dados atual.

## Isso criará a tabela EmployeeDocuments, que será mapeada para uma pasta no sistema de arquivos chamada EmployeeFiles.


### faça um select 

<span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: rgb(232, 232, 232); font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;"><br></span>

In [19]:
SELECT * FROM  dbo.EmployeeDocuments

(2 rows affected)

Total execution time: 00:00:05.633

stream_id,file_stream,name,path_locator,parent_path_locator,file_type,cached_file_size,creation_time,last_write_time,last_access_time,is_directory,is_offline,is_hidden,is_readonly,is_archive,is_system,is_temporary
e66ecfd0-bd76-ef11-8cce-70a8d3d10a05,...,dataset.xlsx,0xFE837646F89419AFD851525D75B112F9E8993466E0,NULL,xlsx,4902764,2024-09-19 16:31:50.5926502 -03:00,2024-09-19 16:31:50.5926502 -03:00,2024-09-19 16:31:50.5926502 -03:00,0,0,0,0,1,0,0


## 6. Acessar a FileTable via Windows Explorer:

Após a criação da FileTable, o SQL Server disponibiliza automaticamente um caminho UNC (Universal Naming Convention) para acessar os arquivos.

O caminho para acessar a pasta será algo como:

\\<SQLServerName>\<SQLInstanceName>\FileTables\EmployeeFiles

Você pode navegar até essa pasta no Windows Explorer e arrastar ou soltar arquivos diretamente nela. Qualquer arquivo que você mover para essa pasta será automaticamente inserido no banco de dados SQL Server como um registro na FileTable 

EmployeeDocuments.

## <span style="font-size: 14pt; font-family: Arial, sans-serif; color: rgb(0, 0, 0); background-color: rgb(232, 232, 232); font-weight: 700; font-style: italic; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Adicionar um arquivo via T-SQL:</span>

<span style="font-size: 13pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Você também pode adicionar um novo arquivo na FileTable usando comandos T-SQL. Aqui está um exemplo de como isso seria feito, inserindo um arquivo no formato binário diretamente:</span>

<span style="font-size: 13pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;"><br></span>

In [15]:
INSERT INTO EmployeeDocuments (name, file_stream)
VALUES ('dataset.xlsx', 
        (SELECT * FROM OPENROWSET(BULK N'D:\wesley\SQLDemo\Arquivos\dataset.xlsx', SINGLE_BLOB) AS Document));



: Msg 2627, Level 14, State 1, Line 1
Violation of UNIQUE KEY constraint 'UQ__Employee__A236CBB36ACA43B4'. Cannot insert duplicate key in object 'dbo.EmployeeDocuments'. The duplicate key value is (<NULL>, dataset.xlsx).

The statement has been terminated.

Total execution time: 00:00:00.036

In [22]:
INSERT INTO EmployeeDocuments (name, file_stream)
VALUES ('ContraChequeJuizesPoderJudiciario.csv', 
        (SELECT * FROM OPENROWSET(BULK N'D:\wesley\SQLDemo\Arquivos\ContraChequeJuizesPoderJudiciario.csv', SINGLE_BLOB) AS Document));

: Msg 2627, Level 14, State 1, Line 1
Violation of UNIQUE KEY constraint 'UQ__Employee__A236CBB36ACA43B4'. Cannot insert duplicate key in object 'dbo.EmployeeDocuments'. The duplicate key value is (<NULL>, ContraChequeJuizesPoderJudiciario.csv).

The statement has been terminated.

Total execution time: 00:00:00.165

#### <span style="font-size: 13pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">7. Manipulação dos arquivos via T-SQL:</span>

<span style="font-size: 14pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Agora que você tem a FileTable funcionando, pode começar a consultar, adicionar e modificar arquivos diretamente pelo SQL Server.</span>

##### <span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">a) Consultar arquivos armazenados:</span>

<span style="font-size: 14pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Para consultar informações dos arquivos armazenados, você pode executar uma query SQL simples:</span>

<span style="font-size: 14pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;"><br></span>

In [23]:
SELECT name, file_stream, creation_time, last_write_time
FROM EmployeeDocuments;

(2 rows affected)

Total execution time: 00:00:05.964

name,file_stream,creation_time,last_write_time
dataset.xlsx,...,2024-09-19 16:31:50.5926502 -03:00,2024-09-19 16:31:50.5926502 -03:00
,0x,NULL,NULL


## Essa consulta retorna informações como o nome do arquivo, o conteúdo binário do arquivo (file\_stream), a data de criação e a última modificação.

#### <span style="font-size: 13pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">8. Gerenciar Documentos via Sistema e Banco:</span>

<span style="font-size: 14pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Agora, sua empresa pode:</span>

- <span style="font-size: 14pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Arrastar e soltar documentos</span> <span style="font-size: 14pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;"> na pasta associada à FileTable, e eles serão automaticamente inseridos no banco de dados.</span>
    
- <span style="font-size: 14pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Consultar e manipular os documentos</span> <span style="font-size: 14pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;"> diretamente via T-SQL.</span>
    
- <span style="font-size: 14pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Controlar permissões</span> <span style="font-size: 14pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;"> de acesso aos documentos tanto via SQL Server quanto via permissões NTFS.</span>
    

####   

<span style="font-size: 14pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;"><br></span>

#### <span style="font-size: 13pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">9. Backup e Restauração:</span>

<span style="font-size: 14pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Todos os arquivos armazenados no FileTable fazem parte do banco de dados. Isso significa que eles podem ser incluídos nos backups regulares do SQL Server, garantindo que todos os documentos estejam seguros.</span>

### <span style="font-size: 15pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Benefícios do FileTable nesse cenário:</span>

- <span style="font-size: 14pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Centralização</span><span style="font-size: 14pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">: Todos os arquivos da empresa estão centralizados no SQL Server, facilitando o backup e a recuperação.</span>
    
- <span style="font-size: 14pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Acesso Simples</span><span style="font-size: 14pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">: Os funcionários podem acessar e manipular documentos facilmente, como fariam com qualquer outro arquivo no sistema de arquivos.</span>
    
- <span style="font-size: 14pt; background-color: transparent; font-weight: 700; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">Relatórios</span><span style="font-size: 14pt; background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; text-wrap: wrap;">: É fácil integrar a FileTable com outros dados da empresa, permitindo gerar relatórios detalhados sobre o uso de documentos.</span>

In [25]:
SELECT 
    DB_NAME() AS DatabaseName,
    physical_name AS FilestreamDirectory
FROM 
    sys.database_files
WHERE 
    type_desc = 'FILESTREAM';


(1 row affected)

Total execution time: 00:00:00.012

DatabaseName,FilestreamDirectory
FileStreamDocumentManagement,D:\wesley\SQLDemo\DocumentManagement_FS


In [30]:
SELECT stream_id,name
FROM dbo.EmployeeDocuments WHEre name ='ContraChequeJuizesPoderJudiciario.csv'

(1 row affected)

Total execution time: 00:00:00.008

stream_id,name
26bbbf6d-be76-ef11-8cce-70a8d3d10a05,ContraChequeJuizesPoderJudiciario.csv
